# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [185]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder.appName("Walmart stock mini project").getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [186]:
w_stocks = spark.read.option("header", "true").option("inferSchema", "true").csv("wasb://sparklab@sbrrsparkhdistorage.blob.core.windows.net/HdiNotebooks/walmart_stock.csv")

#### What are the column names?

In [187]:
w_stocks.schema.names

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [188]:
w_stocks.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

#### Print out the first 5 columns.

In [189]:
w_stocks.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows

#### Use describe() to learn about the DataFrame.

In [190]:
w_stocks.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [191]:
# w_stocks_stats = w_stocks.describe()
# w_stocks_stats.printSchema()

In [192]:
# from pyspark.sql.functions import ceil
# test = w_stocks_stats.select("*",ceil("ID")).show()

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [193]:
hv_ratio = w_stocks.select((w_stocks.High / w_stocks.Volume).alias("HV Ratio")).show()


+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows

#### What day had the Peak High in Price?

In [194]:
peak_high = w_stocks.sort(w_stocks.High.desc()).limit(1)
peak_high = peak_high.select(w_stocks.Date).show()


+-------------------+
|               Date|
+-------------------+
|2015-01-13 00:00:00|
+-------------------+

#### What is the mean of the Close column?

In [195]:
from pyspark.sql.functions import mean, col

close_avg = w_stocks.select(mean(col("Close"))).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+

#### What is the max and min of the Volume column?

In [196]:
from pyspark.sql.functions import max, min
vol_stats = w_stocks.select(max(col("Volume")), min(col("Volume"))).show()


+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+

#### How many days was the Close lower than 60 dollars?

In [197]:
close = w_stocks.filter(w_stocks.Close < 60)
close.count()


81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [198]:
high_80 =  w_stocks.filter(w_stocks.High > 80)
high_80_count = high_80.count()
total_days = w_stocks.count()
print(high_80_count / total_days * 100)

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [199]:
pearson = w_stocks.stat.corr("High", "Volume")
print(pearson)


-0.3384326061737161

#### What is the max High per year?

In [200]:
from pyspark.sql.functions import year

years_highs = w_stocks.select((year("Date").alias("Year")), "High")
years_high_max = years_highs.groupBy("Year").agg({"High": "max"}).show()


+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [201]:
from pyspark.sql.functions import month

month_closes = w_stocks.select((month("Date").alias("Month")), "Close")
month_close_avg = month_closes.groupBy("Month").agg({"Close": "avg"}).sort("Month").show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

# Great Job!